In [1]:
%load_ext autoreload
%autoreload 2

In [69]:
import os
import argparse
import numpy as np
import json

import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import networkx as nx
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt

from minimal.arch import Generator

from minimal.dataset import (
    FloorplanGraphDataset,
    floorplan_collate_fn,
    reader
)

from minimal.utils import (
    init_input,
    draw_masks,
    draw_graph
)

%matplotlib inline

In [3]:
DATA_PATH = "./data/sample_list.txt"

In [4]:
data_path = DATA_PATH
transform = transforms.Normalize(mean=[0.5], std=[0.5])
target_set = 8
split = "test"

In [5]:
subgraphs = []

In [6]:
with open(data_path, "r") as f1:
    lines = f1.readlines()
    lines = list(map(lambda x: x.strip(), lines))

lines

['data/json/45012.json',
 'data/json/45161.json',
 'data/json/36233.json',
 'data/json/19307.json',
 'data/json/7513.json',
 'data/json/18477.json']

In [7]:
# for line in lines:
#     rms_type, fp_eds, rms_bbs, eds_to_rms, eds_to_rms_tmp = reader(line)
    
#     subgraphs.append([
#         rms_type,
#         rms_bbs,
#         fp_eds,
#         eds_to_rms,
#         eds_to_rms_tmp,
#     ])

In [33]:
# rms_type, fp_eds, rms_bbs, eds_to_rms, eds_to_rms_tmp = reader("data/expr.json")

# rms_type: "room_type" from file
# rms_bbs: (12, 4): "boxes" from file, but centralized
# fp_eds: (54, 4): "edges" from file, but centralized
# eds_to_rms: list[list[int]] "ed_rm" from file
# eds_to_rms_tmp: eds_to_rms but the inner list truncated to length 1
#                 i.e eds_to_rms_tmp[i][0] == eds_to_rms[i][0] for every i

# print(eds_to_rms_tmp)

In [6]:
with open("data/expr.json") as f:
    info = json.load(f)

In [8]:
# read from file
rms_type = info["room_type"]
rms_bbs_j = np.array(info["boxes"])
fp_eds_j = np.array(info["edges"])[:, :4]
eds_to_rms = info["ed_rm"]

In [70]:
# # Count rooms that are not equal to 17 (interior_door)
# s_r = 0
# for rmk in range(len(rms_type)):
#     if rms_type[rmk] != 17:
#         s_r = s_r + 1

# print(f"{s_r = }")

In [9]:
# convert bounding boxes from range [0,256] to range [0,1]
rms_bbs_01 = rms_bbs_j / 256.0
fp_eds_01 = fp_eds_j / 256.0

In [11]:
# discard last 2 number of fp_eds
# new shape: (*, 4)
print(fp_eds_01.shape)

(54, 4)


In [12]:
# (R, 2)
rooms_top_lefts = rms_bbs_01[:, :2]
rooms_btm_rights = rms_bbs_01[:, 2:]

# top left of the bounding box of the masks
global_tl = np.min(rooms_top_lefts, axis=0)
# bottom right of the bounding box of the masks
global_br = np.max(rooms_btm_rights, axis=0)

In [17]:
# Make the center of the overall bounding box (0.5, 0.5)
# i.e centralize the floor plan 
shift = (global_tl + global_br) / 2.0 - 0.5
rms_bbs = rms_bbs_01[:, :2] - shift
rms_bbs = rms_bbs_01[:, 2:] - shift
fp_eds = fp_eds_01[:, :2] - shift
fp_eds = fp_eds_01[:, 2:] - shift

In [65]:
triples = []

nodes = rms_type
for k in range(len(nodes)):
    for l in range(k + 1, len(nodes)):
        is_adjacent = False
        for edge in eds_to_rms:
            if (l in edge) and (k in edge):
                is_adjacent = True
                break

        triples.append([k, 1 if is_adjacent else -1, l])

In [67]:
rms_masks = []
im_size = 256
out_size=64
fp_mk = np.zeros((out_size, out_size))

In [70]:
k = 0
eds = [i for i, inner in enumerate(eds_to_rms) if inner[0] == k]

rm_im = Image.new("L", (im_size, im_size))
dr = ImageDraw.Draw(rm_im)

In [79]:
# shape (len(eds), 4)
param0 = fp_eds[eds]
# poly = self.make_sequence(np.array([fp_eds[l][:4] for l in eds]))[
#     0
# ]
# poly = [(im_size * x, im_size * y) for x, y in poly]
# if len(poly) >= 2:
#     dr.polygon(poly, fill="white")
# else:
#     print("Empty room")